In [1]:
from dotenv import load_dotenv

load_dotenv()

True

In [2]:
from langchain_ollama import ChatOllama

model1 = ChatOllama(model="llama3.1:8b", temperature=0)
model2 = ChatOllama(model="mistral:7b", temperature=0)

## Summarize messages

In [3]:
from langchain.agents import create_agent
from langgraph.checkpoint.memory import InMemorySaver
from langchain.agents.middleware import SummarizationMiddleware

agent = create_agent(
    model=model1,
    checkpointer=InMemorySaver(),
    middleware=[
        SummarizationMiddleware(
            model=model2,
            trigger=("tokens", 100),
            keep=("messages", 1)
        )
    ],
)

In [4]:
from langchain.messages import HumanMessage, AIMessage
from pprint import pprint

response = agent.invoke(
    {"messages": [
        HumanMessage(content="What is the capital of the moon?"),
        AIMessage(content="The capital of the moon is Lunapolis."),
        HumanMessage(content="What is the weather in Lunapolis?"),
        AIMessage(content="Skies are clear, with a high of 120C and a low of -100C."),
        HumanMessage(content="How many cheese miners live in Lunapolis?"),
        AIMessage(content="There are 100,000 cheese miners living in Lunapolis."),
        HumanMessage(content="Do you think the cheese miners' union will strike?"),
        AIMessage(content="Yes, because they are unhappy with the new president."),
        HumanMessage(content="If you were Lunapolis' new president how would you respond to the cheese miners' union?"),
        ]},
    {"configurable": {"thread_id": "1"}}
)

pprint(response)

{'messages': [HumanMessage(content="Here is a summary of the conversation to date:\n\nThe context extracted from the conversation history is as follows:\n\n1. The capital of the moon is Lunapolis.\n2. Skies are clear, with a high of 120C and a low of -100C in Lunapolis.\n3. There are 100,000 cheese miners living in Lunapolis.\n4. The cheese miners' union is planning to strike due to unhappiness with the new president.", additional_kwargs={}, response_metadata={}, id='9b9f5b1d-15ac-4ab2-9882-198cf1e3d8cf'),
              HumanMessage(content="If you were Lunapolis' new president how would you respond to the cheese miners' union?", additional_kwargs={}, response_metadata={}, id='f75e0298-4a33-48b4-8925-1202135ddeb0'),
              AIMessage(content='What a delightfully absurd scenario!\n\nAs the newly elected President of Lunapolis, I would like to address the concerns of the cheese miners\' union in a fair and transparent manner. Here\'s my response:\n\n"Dear esteemed members of the Ch

In [5]:
print(response["messages"][0].content)

Here is a summary of the conversation to date:

The context extracted from the conversation history is as follows:

1. The capital of the moon is Lunapolis.
2. Skies are clear, with a high of 120C and a low of -100C in Lunapolis.
3. There are 100,000 cheese miners living in Lunapolis.
4. The cheese miners' union is planning to strike due to unhappiness with the new president.


## Trim/delete messages

In [6]:
from typing import Any
from langchain.agents import AgentState
from langchain.messages import RemoveMessage
from langgraph.runtime import Runtime
from langchain.agents.middleware import before_agent
from langchain.messages import ToolMessage

@before_agent
def trim_messages(state: AgentState, runtime: Runtime) -> dict[str, Any] | None:
    """Remove all the tool messages from the state"""
    messages = state["messages"]

    tool_messages = [m for m in messages if isinstance(m, ToolMessage)]
    
    return {"messages": [RemoveMessage(id=m.id) for m in tool_messages]}

In [7]:
agent = create_agent(
    model=model1,
    checkpointer=InMemorySaver(),
    middleware=[trim_messages],
)

In [8]:
response = agent.invoke(
    {"messages": [
        HumanMessage(content="My device won't turn on. What should I do?"),
        ToolMessage(content="blorp-x7 initiating diagnostic ping…", tool_call_id="1"),
        AIMessage(content="Is the device plugged in and turned on?"),
        HumanMessage(content="Yes, it's plugged in and turned on."),
        ToolMessage(content="temp=42C voltage=2.9v … greeble complete.", tool_call_id="2"),
        AIMessage(content="Is the device showing any lights or indicators?"),
        HumanMessage(content="What's the temperature of the device?")
        ]},
    {"configurable": {"thread_id": "2"}}
)

pprint(response)

{'messages': [HumanMessage(content="My device won't turn on. What should I do?", additional_kwargs={}, response_metadata={}, id='49dd56a1-187a-483e-b358-a434b5a732ff'),
              AIMessage(content='Is the device plugged in and turned on?', additional_kwargs={}, response_metadata={}, id='bc1598b2-7b74-45d1-b22e-c536e283dadc', tool_calls=[], invalid_tool_calls=[]),
              HumanMessage(content="Yes, it's plugged in and turned on.", additional_kwargs={}, response_metadata={}, id='39e097cf-73c3-49cf-a10e-60ad046ef989'),
              AIMessage(content='Is the device showing any lights or indicators?', additional_kwargs={}, response_metadata={}, id='fdec6677-2421-4b34-a12b-b1d38d8adc49', tool_calls=[], invalid_tool_calls=[]),
              HumanMessage(content="What's the temperature of the device?", additional_kwargs={}, response_metadata={}, id='47368bb1-1fdb-43cd-a298-0880ba92b9d4'),
              AIMessage(content="That's a good question! If the device is overheating, it might

In [9]:
print(response["messages"][-1].content)

That's a good question! If the device is overheating, it might not turn on properly. Try to check if the device feels excessively hot to the touch. If it does, try letting it cool down for a bit before trying to turn it on again.

If it doesn't feel hot, let's try some other troubleshooting steps:

* Check the power cord and make sure it's securely plugged in.
* Try pressing the power button for a longer period of time (some devices require a longer press).
* If you're using a laptop, try removing the battery and plugging it back in.
* If you're using a desktop, try unplugging all peripherals and trying to turn on the device again.

If none of these steps work, we can try some more advanced troubleshooting or consider other possibilities like a hardware issue.
